In [1]:
# Load and pre-process *all* BES datasets

In [6]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import gc
import re

from itertools import cycle
from IPython.display import display
import pickle, os

import seaborn as sns

encoding = "ISO-8859-1"

import Jupyter_module_loader

In [7]:
# you should clone this git to this subdirectory (in some directory - I call it BES_analysis - doesn't matter though)

if os.getcwd().split(os.sep)[-1] != 'BES_analysis_code':
    raise Exception("Stop! You're in the wrong directory - should be in 'BES_analysis_code'")

BES_code_folder   = "../BES_analysis_code/" # we should be here!
BES_small_data_files = BES_code_folder + "small data files" + os.sep
if not os.path.exists( BES_small_data_files ):
    os.makedirs( BES_small_data_files )

# we should create these if they don't already exist
BES_data_folder   = "../BES_analysis_data/"
if not os.path.exists( BES_data_folder ):
    os.makedirs( BES_data_folder )

BES_output_folder = "../BES_analysis_output/"
if not os.path.exists( BES_output_folder ):
    os.makedirs( BES_output_folder )
    
BES_file_manifest = pd.read_csv( BES_small_data_files + "BES_file_manifest.csv" )

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [ ]:
import Untitled5
from Untitled5 import process_dataset

In [ ]:
for dataset_name in ['W10_comb']:
# [x for x in BES_file_manifest["Name"] if not ("strings" in x)]:


# [x for x in BES_file_manifest["Name"] if not ("strings" in x)]:
# [x for x in BES_file_manifest["Name"] if not ( ("strings" in x) or ("comb" in x) )]:
# [x for x in BES_file_manifest["Name"] if not ("strings" in x)]: #  and not ("strings" in x)]:
    if dataset_name in BES_file_manifest["Name"].values:
        print(dataset_name)
        process_dataset(dataset_name)

W10_comb


In [5]:
# %debug


In [ ]:
 # copy columns from var_type
var_type_all_datasets = pd.DataFrame()

for dataset_name in [x for x in BES_file_manifest["Name"] if not ("strings" in x)]:
    print(dataset_name)

    data_subfolder = BES_data_folder + dataset_name + os.sep
    
#     var_type = pd.read_hdf( data_subfolder + "var_type.hdf", "var_type")
    var_type = pd.read_csv( data_subfolder + "var_type.csv", encoding = encoding )
#     print(data_subfolder + "var_type.hdf")
    if len( var_type_all_datasets.columns ) == 0:
        var_type_all_datasets = pd.DataFrame(columns = var_type.columns)
    
    var_type_all_datasets = var_type_all_datasets.append(var_type)

# var_type_all_datasets.reset_index(inplace = True)
var_type_all_datasets.rename(columns = {'Unnamed: 0':'variable_name'},inplace = True)    

var_type_all_datasets.to_csv( BES_data_folder + "var_type_all_datasets.csv", encoding = encoding )


In [10]:
## COLUMNS THAT EITHER LACK ALL DATA OR HAVE ACTUAL ERRORS
# check back on these periodically - one assumes they will get fixed!
# maybe tell them about them so that they can?

# {'changeIssue1W9', 'conLeaderLikeW9'}
# these variables appear to have disappeared! Fixed in an updated version?

ignore_list = ['whichPartiesHelped_99W6',
               'partyContactGrnW1',
               'partyContactGrnW2',
               'partyContactGrnW3',
               'reasonNotRegistered_noneW2',               
               'reasonNotRegistered_noneW3',
               'reasonNotRegistered_noneW4',
               'reasonNotRegistered_noneW6',
               'reasonNotRegistered_noneW7',
               'reasonNotRegistered_noneW8',
               'reasonNotRegistered_none',
               'partyContactSNPW1',
               'partyContactSNPW2',
               "locusControlW9",
               "generalElecCertaintyW1", # wave 10 forwards
               "generalElecCertaintyW2",
               "generalElecCertaintyW3",
               "londonMayorVoteW7",
               "fatherNumEmployeesW4",
               "motherNumEmployeesW4",
               "profile_pcon_2010_newW3", # W3_comb: this is parl. constit. ... but by number!
               "euroElectionVoteYoungW2", # W3_comb: all NaNs!
               "profile_GOR_pdlW4", # W4_comb: misnamed selection, probably fixable 
               "participation_111W5", ### -->
               "sharedContentOnline_111W5",
               "sharedContentOnline_99W5", ### <-- W5_comb "Got a lot worse|Got a little worse" doesn't look right (indicator vars?)
               "csplScotRefW3", ### W5_comb: "North East" - just broken!
              ]

#- approveEUW2 'Strongly disapprove|Disapprove|Don't know' - should be "approve|disapprove|don't know"??? NOT SURE (distribution weird)
#- whichPartiesHelped_99W6 - answer set = ["No"]
#- partyContactGrnW1 ... reasonNotRegistered_noneW8 answer set = ["No", "Don't know"]
# -partyContactSNPW1, partyContactSNPW2 - answer set = ["Don't know"]
# -changeIssue1W9|conLeaderLikeW9|locusControlW9 - answer set = ["No formal qualifications"]

In [11]:
var_type_all_datasets = pd.read_csv( BES_data_folder + "var_type_all_datasets.csv", encoding = encoding )

BES_file_manifest = pd.read_csv( BES_small_data_files + "BES_file_manifest.csv" )
ign_out = pd.DataFrame(columns = ["dataset_name","variable_name","dtype","cat_all_strings","cat_list"])
ign_ds = var_type_all_datasets[ var_type_all_datasets["variable_name"].apply(lambda x: x in ignore_list) ]["dataset_name"].unique()

for dataset_name in ign_ds:

    manifest = BES_file_manifest[ BES_file_manifest["Name"] == dataset_name ]

    filename = manifest["Stata_Filename"].values[0]

    data_subfolder = BES_data_folder + dataset_name + os.sep

    BES_Panel = pd.read_stata( data_subfolder + filename)

    w13_igns = var_type_all_datasets[ var_type_all_datasets["variable_name"].apply(lambda x: x in ignore_list) ]\
        [var_type_all_datasets["dataset_name"]==dataset_name]


    for var in w13_igns["variable_name"].values:
        
        dtype = BES_Panel[ var ].dtypes.name
        if (dtype == 'category'):
            cat_all_strings = np.all([isinstance(x,str) for x in BES_Panel[ var ].cat.categories])
            if cat_all_strings:
                cat_list = "|".join(BES_Panel[ var ].cat.categories)
                print(var,":" ,'"'+cat_list+'"' )
            else:
                cat_list = BES_Panel[ var ].cat.categories
                print(var,":" ,cat_list )
        else:
            cat_all_strings = np.nan
            cat_list = (BES_Panel[ var ].dropna().unique())
            print(var,":" , cat_list )
            
        temp_row = pd.DataFrame([[dataset_name,var,dtype,cat_all_strings,cat_list]],
            columns = ["dataset_name","variable_name","dtype","cat_all_strings","cat_list"])            
        ign_out = ign_out.append( temp_row )      

ign_out.reset_index( drop = True , inplace = True )        
ign_out.to_csv( BES_data_folder + "ignore_out.csv", encoding = encoding )

C:\Users\Marios\AppData\Local\conda\conda\envs\BES_analysis\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


partyContactSNPW1 : "Don't know"
partyContactGrnW1 : "No|Don't know"
partyContactSNPW2 : "Don't know"
partyContactGrnW2 : "No|Don't know"
reasonNotRegistered_noneW2 : "No|Don't know"
partyContactGrnW3 : "No|Don't know"
reasonNotRegistered_noneW3 : "No|Don't know"
reasonNotRegistered_noneW4 : "No|Don't know"
reasonNotRegistered_noneW6 : "No|Don't know"
whichPartiesHelped_99W6 : "No"
reasonNotRegistered_noneW7 : "No|Don't know"
reasonNotRegistered_noneW8 : "No|Don't know"
partyContactSNPW1 : "Don't know"
partyContactGrnW1 : "No|Don't know"
generalElecCertaintyW1 : [ 1.]
partyContactSNPW2 : "Don't know"
partyContactGrnW2 : "No|Don't know"
reasonNotRegistered_noneW2 : "No|Don't know"
generalElecCertaintyW2 : [ 1.]
partyContactGrnW3 : "No|Don't know"
reasonNotRegistered_noneW3 : "No|Don't know"
generalElecCertaintyW3 : [ 1.]
reasonNotRegistered_noneW4 : "No|Don't know"
reasonNotRegistered_noneW6 : "No|Don't know"
whichPartiesHelped_99W6 : "No"
londonMayorVoteW7 : Float64Index([1.0], dtype='

In [12]:
ign_out_dataset_list = pd.DataFrame(columns = ["dataset_list","variable_name","dtype","cat_all_strings","cat_list"])
newdf_index = 0
for col in ign_out["variable_name"].unique():

    indices = ign_out[ ign_out["variable_name"]==col ].index
    dataset_list = []
    dataset_list.append( ign_out.loc[ indices[0] ]["dataset_name"] )  
    
    cat_list        = ign_out.loc[ indices[0] ]["cat_list"]
    dtype           = ign_out.loc[ indices[0] ]["dtype"]
    cat_all_strings = ign_out.loc[ indices[0] ]["cat_all_strings"]    
    
    if len(indices) > 1:

        for index in indices[1:len(indices)]:
            if ( not np.all([i==j for i, j in zip(ign_out.loc[ index ]["cat_list"], cat_list)]) ):
                print( col, cat_list, ign_out.loc[ index ]["cat_list"] )
            if ( ign_out.loc[ index ]["dtype"] != dtype ):
                print( col, dtype   , ign_out.loc[ index ]["dtype"] )
            if ( not np.isnan(ign_out.loc[ index ]["cat_all_strings"]) and\
                (ign_out.loc[ index ]["cat_all_strings"] != cat_all_strings) ):
                print( col, cat_all_strings   , ign_out.loc[ index ]["cat_all_strings"] )
                dataset_list.append( ign_out.loc[ index ]["dataset_name"] )

    ign_out_dataset_list.loc[newdf_index] = [dataset_list, col, dtype, cat_all_strings, cat_list ]
    newdf_index = newdf_index + 1
        
        
ign_out_dataset_list.sort_values(by="variable_name").to_csv( BES_data_folder + "ignore_out_stacked_by_dataset.csv", encoding = encoding )

generalElecCertaintyW1 float64 float32
generalElecCertaintyW1 float64 float32
generalElecCertaintyW1 float64 float32
generalElecCertaintyW2 float64 float32
generalElecCertaintyW2 float64 float32
generalElecCertaintyW2 float64 float32
generalElecCertaintyW3 float64 float32
generalElecCertaintyW3 float64 float32


In [13]:
%%time
## Look for disparities in variable catlists in different datasets

# pruned stubs of the variables whose categories we know change over time
Changeable = ["bestPM","generalElectionVote","generalElectionVotePost","generalElectionVoteUnsqueeze","generalElectionVoteSqueeze",
              "localElectionVote","discussantVoteName1","discussantVoteName2","discussantVoteName3","recallVote15",
              "miilabel","EUMIICategory", "expectGoodConductGeneral","partyId","partyIdSqueeze","bestOnMII","votingWish",
              "partyMemberNow","localElectionVote","discussantVoteName1","discussantVoteName2","discussantVoteName3",
              "profile_mpname","discussantVoteName","recallVote","mpName",
              'gor', 'profile_lea', 'profile_oslaua','country_of_residence',
              'approveEU','discussantturnoutName', 'housing'
             ]
# 'housing' keep it out for now to see result!
# discussantturnoutName1,discussantturnoutName2,discussantturnoutName3 - the questions just changed!


# party based one tend to differ by British National Party (BNP) and Scottish National Party (SNP)



var_type_all_stacked_by_dataset = pd.DataFrame(columns = ["variable_name","initial_dataset_name","compared_dataset_name","dtype",
                                                          "type","pruned","first_variable_cat_list","final_cat_list" ] )

newdf_index = 0
var_type_all_datasets = pd.read_csv( BES_data_folder + "var_type_all_datasets.csv", encoding = encoding )
df = var_type_all_datasets
unique_variable_names = list( df["variable_name"].unique() )
for col in unique_variable_names:

    if col in ignore_list:
        continue

        
    indices = df[ df["variable_name"]==col ].index
    dataset_list = []
    init_ds = df.loc[ indices[0] ]["dataset_name"]
    dataset_list.append( init_ds )  
    
    final_cat_list        = df.loc[ indices[0] ]["final_cat_list"]
    vtype                 = df.loc[ indices[0] ]["type"]
    dtype                 = df.loc[ indices[0] ]["dtype"]    
    pruned                = df.loc[ indices[0] ]["pruned"]   

    if pruned in Changeable:
        continue    
    
    if (vtype==6):
        continue
#     print(col, final_cat_list)
#     var_type_all_stacked_by_dataset.loc[newdf_index] = [ col, init_ds, dtype, vtype, pruned, final_cat_list ]
#     newdf_index = newdf_index + 1
    if len(indices) > 1:

        for index in indices[1:len(indices)]:
#             print(df.loc[ index ]["final_cat_list"])
            cur_ds = df.loc[ index ]["dataset_name"]            
            if isinstance(df.loc[ index ]["final_cat_list"],str) and\
                df.loc[ index ]["final_cat_list"] != final_cat_list:
#                 print( col, "final_cat_list: ",init_ds, final_cat_list, cur_ds , df.loc[ index ]["final_cat_list"] )

#             if (not isinstance( df.loc[ index ]["dataset_specific_hardcoded_fix"] , str))\
#                 and (not isinstance( df.loc[ indices[0] ]["dataset_specific_hardcoded_fix"] , str))\
#                 and (df.loc[ index ]["dtype"]          != dtype):
#                 print( col, "dtype: " , init_ds, dtype, cur_ds , df.loc[ index ]["dtype"] )
#             if df.loc[ index ]["type"] != vtype:
#                 print( col, "type: "  , init_ds, vtype, cur_ds , df.loc[ index ]["type"] )
#             if isinstance(df.loc[ index ]["pruned"],str) and\
#                 df.loc[ index ]["pruned"] != pruned:
#                 print( col, "pruned: ", init_ds, pruned,cur_ds , df.loc[ index ]["pruned"] )                
                dataset_list.append( cur_ds )

                var_type_all_stacked_by_dataset.loc[newdf_index] = [col,init_ds,cur_ds,dtype,vtype,pruned,final_cat_list,df.loc[ index ]["final_cat_list"]]
                newdf_index = newdf_index + 1
        
# var_type_all_stacked_by_dataset.drop(axis=)        
var_type_all_stacked_by_dataset.to_csv( BES_data_folder + "var_type_all_stacked_by_dataset.csv", encoding = encoding )

Wall time: 1min 1s


In [14]:
var_type_all_stacked_by_dataset
#.drop(["Unnamed: 0"],axis=1)  

,variable_name,initial_dataset_name,compared_dataset_name,dtype,type,pruned,first_variable_cat_list,final_cat_list


In [15]:
# var_type_all_datasets.columns[0]

In [16]:
var_type_all_stacked_by_dataset["pruned"].unique() 

array([], dtype=object)

In [17]:
var_type_all_datasets[ var_type_all_datasets["typos"].notnull() ]

,Unnamed: 0,variable_name,dataset_name,dtype,cat_all_strings,type,pruned,original_cat_list,renamed_cat_list,reordered_cat_list,final_cat_list,dataset_specific_hardcoded_fix,numerical_dont_knows,weasel_words,typos
267,267,anyUni,W13_only,category,True,1,anyUni,"No, I have never attended higher education|Yes...",NaN,NaN,"No, I have never attended higher education|Yes...",NaN,NaN,Don't know,"No, I have never attended higher education|Yes..."
289,289,profile_religion,W13_only,category,True,3,profile_religion,"No, I do not regard myself as belonging to any...","No, I do not regard myself as belonging to any...",NaN,"No, I do not regard myself as belonging to any...",NaN,NaN,Prefer not to say,"No, I do not regard myself as belonging to any..."
292,292,profile_gross_personal,W13_only,category,True,1,profile_gross_personal,"under Â£5,000 per year|Â£5,000 to Â£9,999 per ...",NaN,NaN,"under £5,000 per year|£5,000 to £9,999 per yea...",NaN,NaN,Prefer not to answer|Don't know,"under Â£5,000 per year|Â£5,000 to Â£9,999 per ..."
794,482,mpNameW1,W13_comb,category,True,3,mpName,Gerald Howarth|Richard Shepherd|Graham Brady|N...,NaN,NaN,Gerald Howarth|Richard Shepherd|Graham Brady|N...,NaN,NaN,NaN,Gerald Howarth|Richard Shepherd|Graham Brady|N...
1221,909,finlit1W2,W13_comb,category,True,2,finlit,More than £102|Exactly £102|Less than £102|Don...,NaN,NaN,More than £102|Exactly £102|Less than £102,NaN,NaN,Don't know|Prefer not to say,More than £102|Exactly £102|Less than £102|Don...
1222,910,finlit2W2,W13_comb,category,True,2,finlit,More than today|Exactly the same as today|Less...,NaN,NaN,More than today|Exactly the same as today|Less...,NaN,NaN,Don't know|Prefer not to say,More than today|Exactly the same as today|Less...
1223,911,finlit3W2,W13_comb,category,True,2,finlit,An individual share in a company|A portfolio o...,NaN,An individual share in a company|A portfolio o...,An individual share in a company|The risk is t...,NaN,NaN,Prefer not to say|Don't know,An individual share in a company|A portfolio o...
2221,1909,finlit1W4,W13_comb,category,True,2,finlit,More than £102|Exactly £102|Less than £102|Don...,NaN,NaN,More than £102|Exactly £102|Less than £102,NaN,NaN,Don't know|Prefer not to say,More than £102|Exactly £102|Less than £102|Don...
2222,1910,finlit2W4,W13_comb,category,True,2,finlit,More than today|Exactly the same as today|Less...,NaN,NaN,More than today|Exactly the same as today|Less...,NaN,NaN,Don't know|Prefer not to say,More than today|Exactly the same as today|Less...
2223,1911,finlit3W4,W13_comb,category,True,2,finlit,An individual share in a company|A portfolio o...,NaN,An individual share in a company|A portfolio o...,An individual share in a company|The risk is t...,NaN,NaN,Prefer not to say|Don't know,An individual share in a company|A portfolio o...


In [18]:
# 368 different final_cat_lists

In [19]:
# var_type_all_stacked_by_dataset = pd.DataFrame(columns = ["variable_name","initial_dataset_name","compared_dataset_name","dtype",
#                                                           "type","pruned","first_variable_cat_list","final_cat_list" ] )

In [20]:
# var_type_all_stacked_by_dataset

In [21]:
# var_type_all_stacked_by_dataset.loc[newdf_index] = [col,init_ds,cur_ds,dtype,vtype,pruned,final_cat_list,df.loc[ index ]["final_cat_list"]]